### Prompt Template - Predefined instructions for models and that model will work a/c to that Prompt Template

In [10]:
import os 
from dotenv import load_dotenv
load_dotenv()
groq_api_key= os.getenv("GROQ_API_KEY")

from langchain_groq import ChatGroq
model = ChatGroq(model="gemma2-9b-it", groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x74cd095ec0d0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x74cd095ed150>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()

    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)
config = {"configurable": {"session_id": "chat1"}}

In [12]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI assistant. Answetr all the questions to the best of your abiility"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model

In [13]:
from langchain_core.messages import HumanMessage
chain.invoke({"messages": [HumanMessage(content="Hi, my name is Piyush")]})


AIMessage(content="Hello Piyush!  \n\nIt's nice to meet you. 😊 \n\nIs there anything I can help you with today?  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 37, 'total_tokens': 70, 'completion_time': 0.06, 'prompt_time': 0.000392859, 'queue_time': 0.01403659, 'total_time': 0.060392859}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-f1ea5612-4f89-491d-bae7-ae43bf8f3431-0', usage_metadata={'input_tokens': 37, 'output_tokens': 33, 'total_tokens': 70})

In [14]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history)
response= with_message_history.invoke(
    [HumanMessage(content="Hi, my name is Piyush")],
    config=config
)
response.content

"Hi Piyush! \n\nIt's nice to meet you.  \n\nIs there anything I can help you with today? 😊  \n\n"

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a HGe,pgul AI assistant. Answetr all the questions to the best of your abiility in {language}"),
        MessagesPlaceholder(variable_name="messages")
    ]
)
chain = prompt | model


In [16]:
response = chain.invoke({"messages": [HumanMessage(content="Hi, My name is Piyush")], "language": "Hindi"})
response.content

'नमस्ते Piyush! \n\nमुझे खुशी है कि आपने मुझसे बात करना चाहा। आप मुझसे जो भी पूछना चाहें, मैं अपनी पूरी क्षमता के साथ उत्तर देने की कोशिश करूँगा।  \n\nआपके कोई प्रश्न हैं? 😊 \n'

In [17]:
with_message_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")
response= with_message_history.invoke(
    {"messages": [HumanMessage(content="Hi, my name is Piyush")], "language": "Hindi"},
    config=config
)
response

AIMessage(content='क्या हुआ, Piyush?  मुझे बताओ, मैं कैसे मदद कर सकता हूँ?  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 91, 'total_tokens': 116, 'completion_time': 0.045454545, 'prompt_time': 0.002570522, 'queue_time': 0.011184618, 'total_time': 0.048025067}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-b76e0508-df32-4153-aa7c-c5823fe8709a-0', usage_metadata={'input_tokens': 91, 'output_tokens': 25, 'total_tokens': 116})

# Manging the conversation history

In [ ]:
from langchain_core.messages import SystemMessage,AIMessage, trim_messages, HumanMessage

trimmer = trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"  
)
messages = [
    SystemMessage(content="You are a helpful AI assistant"),
    HumanMessage(content="Hi, my name is Piyush and I am a software engineer"),
    AIMessage(content="HIII!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="Nice")
]
trimmer.invoke(messages)

In [19]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
chain = (
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
)
response=chain.invoke(
    {
        "messages": messages + [HumanMessage(content="What flavour of ice cream i like?")],
        "language": "English"
    }
)
response.content

"You said you like vanilla ice cream! \n\nIs there another flavor you're curious about?  🍦 😊 \n"

In [24]:
with_message_history= RunnableWithMessageHistory(chain, get_session_history, input_messages_key="messages")

config = {"configurable": {"session_id": "chat3"}}

response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="What flavour of ice cream i like?")],
        "language": "English"
    },
    config=config
)
response.content

"You said you like vanilla ice cream!  🍦  \n\nIs there another flavor you're wondering about? \n"